In [4]:
import numpy as np
from pyquil import Program, get_qc
from pyquil.gates import H
from pyquil.quil import DefGate
from pyquil.api import WavefunctionSimulator 

Let's work with three qubits:

In [5]:
qv = get_qc('3q-qvm')

Now let's drive an equal superposition of all possible states, using the Hadamard gate. Then let's peer into what the resultant wavefunction looks like.

In [6]:
p = Program(H(0), H(1), H(2))
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn)

(0.3535533906+0j)|000> + (0.3535533906+0j)|001> + (0.3535533906+0j)|010> + (0.3535533906+0j)|011> + (0.3535533906+0j)|100> + (0.3535533906+0j)|101> + (0.3535533906+0j)|110> + (0.3535533906+0j)|111>


Remember that since we are using 3 qubits, we expect $2^3 = 8$ possible states, with a relative normalization $\frac{1}{\sqrt{2^3}} = 0.3535533906$. So this state looks exactly as we expect. Now let's define our oracle. For this oracle, I'm going to choose state number 6 to be of interest. Remember that our matrix must be $8x8$ since we are working with three qubits ($16x16$ if we were working with four qubits, etc.).

In [11]:
oracle = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0],                                                                                
    [0, 1, 0, 0, 0, 0, 0, 0],                                                                                
    [0, 0, 1, 0, 0, 0, 0, 0],                                                                                
    [0, 0, 0,-1, 0, 0, 0, 0],                                                                                
    [0, 0, 0, 0, 1, 0, 0, 0],                                                                                
    [0, 0, 0, 0, 0, 1, 0, 0],                                                                                
    [0, 0, 0, 0, 0, 0, 1, 0],                                                                                
    [0, 0, 0, 0, 0, 0, 0, 1]
])
oracle_definition = DefGate("oracle", oracle)
ORACLE = oracle_definition.get_constructor()

Now let's add our oracle operator into our program, along with its definition so that pyquil knows how to use it. We want oracle to act on all three qubits, so it's pyquil instruction will be $\textsf{ORACLE(0, 1, 2)}$

In [12]:
p += oracle_definition
p += ORACLE(0, 1, 2)
print(p)

DEFGATE oracle:
    1, 0, 0, 0, 0, 0, 0, 0
    0, 1, 0, 0, 0, 0, 0, 0
    0, 0, 1, 0, 0, 0, 0, 0
    0, 0, 0, 1, 0, 0, 0, 0
    0, 0, 0, 0, 1, 0, 0, 0
    0, 0, 0, 0, 0, -1, 0, 0
    0, 0, 0, 0, 0, 0, 1, 0
    0, 0, 0, 0, 0, 0, 0, 1

DEFGATE grover-diffusion:
    -0.75, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25
    0.25, -0.75, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25
    0.25, 0.25, -0.75, 0.25, 0.25, 0.25, 0.25, 0.25
    0.25, 0.25, 0.25, -0.75, 0.25, 0.25, 0.25, 0.25
    0.25, 0.25, 0.25, 0.25, -0.75, 0.25, 0.25, 0.25
    0.25, 0.25, 0.25, 0.25, 0.25, -0.75, 0.25, 0.25
    0.25, 0.25, 0.25, 0.25, 0.25, 0.25, -0.75, 0.25
    0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, -0.75

DEFGATE oracle:
    1, 0, 0, 0, 0, 0, 0, 0
    0, 1, 0, 0, 0, 0, 0, 0
    0, 0, 1, 0, 0, 0, 0, 0
    0, 0, 0, -1, 0, 0, 0, 0
    0, 0, 0, 0, 1, 0, 0, 0
    0, 0, 0, 0, 0, 1, 0, 0
    0, 0, 0, 0, 0, 0, 1, 0
    0, 0, 0, 0, 0, 0, 0, 1

H 0
H 1
H 2
oracle 0 1 2
grover-diffusion 0 1 2
oracle 0 1 2



/home/donald/miniconda3/lib/python3.6/site-packages/pyquil/quil.py:166: UserWarning: Gate oracle has already been defined in this program
  .format(instruction.name))


Next we need to construct the Grover diffusion operator. This operator is $D = H \times (-I) \times H$, where $(-I)$ is an inversion about zero. I'm going to use the already-multiplied form, which can be found explicitly here:  htps://people.eecs.berkeley.edu/~vazirani/f04quantum/notes/lec10/lec11.pdf Let's define the Grover diffusion operator for our state (with $N = 2^n = 8$), and then add it to our program.

In [13]:
diffusion = np.array([
    [1/4-1, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4],                                                             
    [1/4, 1/4-1, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4],                                                             
    [1/4, 1/4, 1/4-1, 1/4, 1/4, 1/4, 1/4, 1/4],                                                             
    [1/4, 1/4, 1/4, 1/4-1, 1/4, 1/4, 1/4, 1/4],                     
    [1/4, 1/4, 1/4, 1/4, 1/4-1, 1/4, 1/4, 1/4],              
    [1/4, 1/4, 1/4, 1/4, 1/4, 1/4-1, 1/4, 1/4],     
    [1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4-1, 1/4],                                                             
    [1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4, 1/4-1]  
])
diffusion_definition = DefGate("grover-diffusion", diffusion)
DIFFUSE = diffusion_definition.get_constructor()
p += diffusion_definition
p += DIFFUSE(0, 1, 2)
print(p)

DEFGATE oracle:
    1, 0, 0, 0, 0, 0, 0, 0
    0, 1, 0, 0, 0, 0, 0, 0
    0, 0, 1, 0, 0, 0, 0, 0
    0, 0, 0, 1, 0, 0, 0, 0
    0, 0, 0, 0, 1, 0, 0, 0
    0, 0, 0, 0, 0, -1, 0, 0
    0, 0, 0, 0, 0, 0, 1, 0
    0, 0, 0, 0, 0, 0, 0, 1

DEFGATE grover-diffusion:
    -0.75, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25
    0.25, -0.75, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25
    0.25, 0.25, -0.75, 0.25, 0.25, 0.25, 0.25, 0.25
    0.25, 0.25, 0.25, -0.75, 0.25, 0.25, 0.25, 0.25
    0.25, 0.25, 0.25, 0.25, -0.75, 0.25, 0.25, 0.25
    0.25, 0.25, 0.25, 0.25, 0.25, -0.75, 0.25, 0.25
    0.25, 0.25, 0.25, 0.25, 0.25, 0.25, -0.75, 0.25
    0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, -0.75

DEFGATE oracle:
    1, 0, 0, 0, 0, 0, 0, 0
    0, 1, 0, 0, 0, 0, 0, 0
    0, 0, 1, 0, 0, 0, 0, 0
    0, 0, 0, -1, 0, 0, 0, 0
    0, 0, 0, 0, 1, 0, 0, 0
    0, 0, 0, 0, 0, 1, 0, 0
    0, 0, 0, 0, 0, 0, 1, 0
    0, 0, 0, 0, 0, 0, 0, 1

DEFGATE grover-diffusion:
    -0.75, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25
    0.25, -

/home/donald/miniconda3/lib/python3.6/site-packages/pyquil/quil.py:166: UserWarning: Gate grover-diffusion has already been defined in this program
  .format(instruction.name))


Now, let's think about what we should expect of the wavefunction resulting from this program. We should expect that the program initializes an even superposition state of three qubits (therefore 8 states superimposed), that the oracle inverts the phase of one of the states by $180^\circ$ (in this case the sixth state), and that the entire state is inverted in a special way so that the inversion constructively interferes with the state the oracle selected but destructively interfered with the states the oracle did not select. So what should the output look like? Take a look:

In [14]:
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn)

(-0.0883883476+0j)|000> + (-0.0883883476+0j)|001> + (-0.0883883476+0j)|010> + (-0.0883883476+0j)|011> + (-0.0883883476+0j)|100> + (0.9722718241+0j)|101> + (-0.0883883476+0j)|110> + (-0.0883883476+0j)|111>


Successfully, our program increased the amplitude of the oracle's state and decreased the amplitudes of the other states. Therefore when this wavefunction is measured, the qubits will have a higher probability of being in the configuration $|101>$ than being in any other configuration. The more we iterate over Grover's algorithm, the higher the oracle state's amplitude will be. 